In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmplot as gp

gmap = gp.GoogleMapPlotter(40.7128, -74.0060, 6)


In [2]:
data = pd.read_csv("/Users/samkit/Downloads/SocialNetwork/Project/Data/201801_citibikenyc_tripdata.csv")
data.head()
data_backup = data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718990 entries, 0 to 718989
Data columns (total 16 columns):
tripduration               718990 non-null int64
starttime                  718990 non-null object
stoptime                   718990 non-null object
start station id           718990 non-null int64
start station name         718990 non-null object
start station latitude     718990 non-null float64
start station longitude    718990 non-null float64
end station id             718990 non-null int64
end station name           718990 non-null object
end station latitude       718990 non-null float64
end station longitude      718990 non-null float64
bikeid                     718990 non-null int64
name_localizedValue0       718990 non-null object
usertype                   718990 non-null object
birth year                 692847 non-null float64
gender                     718990 non-null int64
dtypes: float64(5), int64(5), object(6)
memory usage: 87.8+ MB


In [4]:
data.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,7.189900e+05,718990.000000,718990.000000,718990.000000,718990.000000,718990.000000,718990.000000,718990.000000,692847.000000,718990.000000
mean,7.566194e+02,1469.512583,40.737894,-73.982934,1460.942601,40.737620,-73.983203,26078.064018,1978.096792,1.169973
std,9.573695e+03,1398.624562,0.044702,0.018219,1396.756964,0.044525,0.018273,6311.929991,12.118001,0.472665
min,6.100000e+01,72.000000,40.646538,-74.017134,72.000000,40.646538,-74.033552,14529.000000,1885.000000,0.000000
25%,3.190000e+02,377.000000,40.719079,-73.994685,376.000000,40.719105,-73.994758,19558.000000,1969.000000,1.000000
50%,5.100000e+02,497.000000,40.739355,-73.985649,497.000000,40.739323,-73.986580,28538.000000,1981.000000,1.000000
75%,8.420000e+02,3170.000000,40.756687,-73.974987,3169.000000,40.756405,-73.975232,31571.000000,1988.000000,1.000000
max,4.421929e+06,3664.000000,45.506364,-73.568906,3664.000000,45.506364,-73.568906,33522.000000,2002.000000,2.000000


Shape of the data.

In [5]:
data.shape

(718990, 16)

To check the presence of null values.

In [6]:
data.isnull().sum()  

tripduration                   0
starttime                      0
stoptime                       0
start station id               0
start station name             0
start station latitude         0
start station longitude        0
end station id                 0
end station name               0
end station latitude           0
end station longitude          0
bikeid                         0
name_localizedValue0           0
usertype                       0
birth year                 26143
gender                         0
dtype: int64

In [7]:
data['age']=2018-data['birth year']

In [8]:
figure, axes = plt.subplots(nrows=6, ncols=1, figsize=(15,15))

i = 1
for col in data.columns:
    if data[col].dtype == np.float64 or data[col].dtype == np.int64:
        ax = plt.subplot(6, 2, i)
        data[col].hist(bins=50)
        ax.set_title(col)
        i += 1

figure.tight_layout()
plt.show()

To remove strange values

In [10]:
print("Initial rows= ",data.shape)
data = data[data['tripduration'] <= 24*60*60*10]
print("Final rows=   ",data.shape)

Initial rows=  (718979, 17)
Final rows=    (718979, 17)


In [14]:
def toCheckUnique(startstation,endstation):
    a = len(data['start station id'].unique())
    b = len(data[['start station id', startstation]].drop_duplicates())
    c = len(data['end station id'].unique())
    d = len(data[['end station id', endstation]].drop_duplicates())

    return a == b and c == d
print("To check if one-to-one mapping exists")
print("For station name      :",toCheckUnique('start station name','end station name'))
print("For station Latitude  :",toCheckUnique('start station latitude','end station latitude'))
print("For station Longitude :",toCheckUnique('start station longitude','end station longitude'))

To check if one-to-one mapping exists
For station name      : True
For station Latitude  : True
For station Longitude : True


In [27]:
gmap.plot(data['start station latitude'], data['start station longitude'], 'cornflowerblue', edge_width=10)
gmap.scatter(data['start station latitude'], data['start station longitude'], '#FF6666', edge_width=10)


In [28]:
gmap.draw("my_map.html")

In [29]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

t1 = data[['start station id', 'start station name', 'start station latitude', 'start station longitude']] \
            .drop_duplicates().rename(columns = {'start station id':'station id', \
                                                 'start station name':'station name', \
                                                 'start station latitude':'station latitude', 
                                                 'start station longitude': 'station longitude'})
t2 = data[['end station id', 'end station name', 'end station latitude', 'end station longitude']] \
        .drop_duplicates().rename(columns = {'end station id':'station id', \
                                             'end station name':'station name', \
                                             'end station latitude':'station latitude', \
                                             'end station longitude': 'station longitude'})
df_loc = pd.concat([t1, t2]).drop_duplicates()

fig, ax = plt.subplots(figsize=(15,15))

# determine range to print based on min, max lat and lon of the data
lat = list(df_loc['station latitude'])
lon = list(df_loc['station longitude'])
text = list(df_loc['station id'])
margin = 0.01 # buffer to add to the range
lat_min = min(lat) - margin
lat_max = max(lat) + margin
lon_min = min(lon) - margin
lon_max = max(lon) + margin

# create map using BASEMAP
m = Basemap(llcrnrlon=lon_min,
            llcrnrlat=lat_min,
            urcrnrlon=lon_max,
            urcrnrlat=lat_max,
            lat_0=(lat_max - lat_min)/2,
            lon_0=(lon_max - lon_min)/2,
            projection='lcc',
            resolution = 'f',)

m.drawcoastlines()
m.fillcontinents(lake_color='aqua')
m.drawmapboundary(fill_color='aqua')
m.drawrivers()

# convert lat and lon to map projection coordinates
lons, lats = m(lon, lat)

# plot points as red dots
ax.scatter(lons, lats, marker = 'o', color='r', zorder=5, alpha=0.6)
for i in range(df_loc.shape[0]):
    plt.text(lons[i], lats[i], text[i])
plt.show()



ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [35]:
data['starttime'] = pd.to_datetime(data['starttime'], format='%Y-%m-%d %H:%M:%S')
data['stoptime'] =pd.to_datetime(data['stoptime'], format='%Y-%m-%d %H:%M:%S')
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 718979 entries, 0 to 718989
Data columns (total 19 columns):
tripduration               718979 non-null int64
starttime                  718979 non-null datetime64[ns]
stoptime                   718979 non-null datetime64[ns]
start station id           718979 non-null int64
start station name         718979 non-null object
start station latitude     718979 non-null float64
start station longitude    718979 non-null float64
end station id             718979 non-null int64
end station name           718979 non-null object
end station latitude       718979 non-null float64
end station longitude      718979 non-null float64
bikeid                     718979 non-null int64
name_localizedValue0       718979 non-null object
usertype                   718979 non-null object
birth year                 692840 non-null float64
gender                     718979 non-null int64
age                        692840 non-null float64
start_seg             

In [49]:
t1 = data[['start station id', 'start station name', 'start station latitude', 'start station longitude']] \
            .drop_duplicates().rename(columns = {'start station id':'station id', \
                                                 'start station name':'station name', \
                                                 'start station latitude':'station latitude', 
                                                 'start station longitude': 'station longitude'})
t2 = data[['end station id', 'end station name', 'end station latitude', 'end station longitude']] \
        .drop_duplicates().rename(columns = {'end station id':'station id', \
                                             'end station name':'station name', \
                                             'end station latitude':'station latitude', \
                                             'end station longitude': 'station longitude'})
df_loc = pd.concat([t1, t2]).drop_duplicates()
print(len(df_loc))

767


In [31]:
def gen_time_segment(dt):
    if dt.minute < 30:
        minute = "%02d" % 0
    else:
        minute = "%02d" % 30
    return "{}-{}-{} {}:{}".format(dt.year, dt.month, dt.day, dt.hour, minute)

data['start_seg'] = [gen_time_segment(dt) for dt in data['starttime']]
data['stop_seg'] = [gen_time_segment(dt) for dt in data['stoptime']]

data[['start station id', 'starttime', 'start_seg', 'end station id', 'stoptime', 'stop_seg']].head()

,start station id,starttime,start_seg,end station id,stoptime,stop_seg
0,315,2018-01-01 00:01:51,2018-1-1 0:00,259,2018-01-01 00:05:07,2018-1-1 0:00
1,3224,2018-01-01 00:02:44,2018-1-1 0:00,470,2018-01-01 00:06:11,2018-1-1 0:00
2,386,2018-01-01 00:03:15,2018-1-1 0:00,2008,2018-01-01 00:13:28,2018-1-1 0:00
3,466,2018-01-01 00:06:44,2018-1-1 0:00,325,2018-01-01 00:12:59,2018-1-1 0:00
4,438,2018-01-01 00:06:57,2018-1-1 0:00,380,2018-01-01 00:13:40,2018-1-1 0:00


In [44]:
inflow = data[['end station id', 'stop_seg']] \
            .groupby(['end station id', 'stop_seg']) \
            .size().reset_index(name='counts') \
            .rename(columns={'end station id':'station id','stop_seg':'time', 'counts':'in_flow_count'})

In [45]:
outflow = data[['start station id', 'start_seg']] \
            .groupby(['start station id', 'start_seg']) \
            .size().reset_index(name='counts') \
            .rename(columns={'start station id':'station id','start_seg':'time', 'counts':'out_flow_count'})

In [46]:
import itertools

station_id_list = list(df_loc['station id'])

# Create combinations of time series and station ids
time_seg_list = list(pd.date_range("2018-01-01 00:00:00", "2018-01-31 23:30:00", freq="30min"))
template = pd.DataFrame(list(itertools.product(station_id_list, time_seg_list)), \
                        columns=["station id", "time"])

# Merge in/out flow information & Add zeros to missing data according to every time segment
dat = pd.merge(inflow, outflow, on=['station id', 'time'], how='outer')
dat['time'] = pd.to_datetime(dat['time'], format='%Y-%m-%d %H:%M')
dat = dat.merge(template, on=["station id", "time"], how="right").fillna(0)
dat.head()

,station id,time,in_flow_count,out_flow_count
0,72,2018-01-01 11:00:00,2.0,0.0
1,72,2018-01-01 12:00:00,1.0,0.0
2,72,2018-01-01 13:30:00,1.0,1.0
3,72,2018-01-01 16:00:00,1.0,0.0
4,72,2018-01-01 17:30:00,1.0,0.0


In [54]:
dat.head()

,station id,time,in_flow_count,out_flow_count
0,72,2018-01-01 11:00:00,2.0,0.0
1,72,2018-01-01 12:00:00,1.0,0.0
2,72,2018-01-01 13:30:00,1.0,1.0
3,72,2018-01-01 16:00:00,1.0,0.0
4,72,2018-01-01 17:30:00,1.0,0.0


we will be able to improve stock management of bikes using this data

Lets explore distance between each node (stations) of our graph 

In [55]:
print("no of nodes in this graph are {}".format(len(station_id_list)))

no of nodes in this graph are 767


we have used python library GeoPy to calculate distance between two places given their latitude and longitudes. this calculates geodesic distance between two points.

In [61]:
from geopy.distance import vincenty

lat_dic = {}
lon_dic = {}
for index, row in df_loc.iterrows():
    lat_dic[row['station id']] = row['station latitude']
    lon_dic[row['station id']] = row['station longitude']


c = itertools.combinations(station_id_list, 2)

distance = []
dist = 0
dist_sum = 0
count = 0
for stn1, stn2 in c:
        dist = vincenty((lat_dic[stn1], lon_dic[stn1]), (lat_dic[stn2], lon_dic[stn2])).meters
        distance.append([stn1,stn2,dist])
        dist_sum += dist
        count += 1
print("The avg distance between the nodes is :",(dist_sum/count))

The avg distance between the nodes is : 8766.541888396345
